# Extracting Music Data from University of Waterloo
In this lab, we will extract the music dataset provided by [Dave Tompkins](https://cs.uwaterloo.ca/~dtompkin/music/)
Associate Professor at University of Waterloo. To extract the data, we utilized the BeautifulSoup library to scrape music data spanning ten consecutive years (1997–2006). This dataset provided song titles and artist names, which we later used to filter music-related search queries from the AOL search logs.

## Importing the beautifulsoup Library
We import beautifulsoup library to extract, navigate, and manipulate HTML/XML content efficiently.

In [1]:
pip install requests beautifulsoup

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [1]:
import csv
import requests
from bs4 import BeautifulSoup

## Web Scraping Music Data (1997–2006) Using BeautifulSoup
In the code below we iterate through each year, construct the corresponding URL, and send a request to retrieve the webpage content. If the request is successful, it parses the HTML to locate a table containing music information. The script extracts key details such as artist, song title, duration, BPM, year, and genre, ensuring each row has the expected structure before storing the data in a list. Finally, it prints the first 10 retrieved songs, or a message if no data was found.

In [2]:
years = [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006]
all_songs = []

In [3]:
for year in years:
    url = f'https://cs.uwaterloo.ca/~dtompkin/music/year/{year}.html'
    response = requests.get(url)

    # Check if the page is accessible
    if response.status_code != 200:
        print(f"Failed to retrieve data for {year}. Status code: {response.status_code}")
        continue

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table with the class 'music'
    table = soup.find('table', class_='music')

    if not table:
        print(f"No table found for {year}. Skipping...")
        continue

    rows = table.find_all('tr')

    if not rows:
        print(f"No rows found for {year}. Skipping...")
        continue

    # Find the index of the header row to determine the structure
    header = rows[0]
    headers = [th.get_text(strip=True) for th in header.find_all('td')]
    print(f"Headers found: {headers}")  # For debugging
    counter = 0
    # Extract data from each row
    for row in rows[1:]:  # Skipping the header row
        cols = row.find_all('td')

        # Ensure the row contains the expected number of columns
        if len(cols) >= 8:  # Expecting at least 8 columns based on the header structure
            song_info = {
                'artist': cols[1].get_text(strip=True),  # ARTIST
                'song': cols[2].get_text(strip=True),   # TITLE
                'time': cols[3].get_text(strip=True),   # TIME
                'bpm': cols[4].get_text(strip=True),     # BPM
                'year': cols[5].get_text(strip=True),    # YEAR
                'details': cols[8].find('a')['href']  # Link inside the details column
            }

            # Get the link from the details column and scrape the third table
            detail_url = song_info['details']
            counter += 1


            if not detail_url:
                song_info['genre'] = "***"
            detail_url = detail_url.replace("../", "")

            detail_url = f'https://cs.uwaterloo.ca/~dtompkin/music/{detail_url}'

            # If the URL is relative, prepend the base URL
            '''if detail_url.startswith('/'):
                detail_url = detail_url.replace("../", "")
                detail_url = f'https://cs.uwaterloo.ca/~dtompkin/music/{detail_url}'
                print('detail_url', detail_url)'''

            # Request the detail page
            detail_response = requests.get(detail_url)
            if detail_response.status_code == 200:
                detail_soup = BeautifulSoup(detail_response.content, 'html.parser')


                # Find all tables with the class 'simpler'
                simpler_tables = detail_soup.find_all('table', class_='simple')

                if len(simpler_tables) >= 3:
                    second_table = simpler_tables[1]
                    third_table = simpler_tables[2]  # Get the third table
                    rows_in_table_album = second_table.find_all('tr')
                    rows_in_table_genre = third_table.find_all('tr')

                    selected_data_album_name = [cell.get_text(strip=True) for cell in rows_in_table_album[1].find_all('td')]
                    selected_data_album_track = [cell.get_text(strip=True) for cell in rows_in_table_album[2].find_all('td')]


                    # Take the second row if there are two, otherwise the first row
                    if len(rows_in_table_genre) >= 3:
                        selected_row = rows_in_table_genre[2]  # Second row
                    else:
                        selected_row = rows_in_table_genre[1]  # First row

                    # Extract the text or information you need from the selected row
                    selected_data_genre = [cell.get_text(strip=True) for cell in selected_row.find_all('td')]


                    # Optionally, add this information to the song_info dictionary
                    song_info['genre'] = selected_data_genre[1]
                    song_info['album'] = selected_data_album_name[1]
                    song_info['track num'] = selected_data_album_track[1]

                else:
                    print(f"Could not find the third table for {song_info['song']}.")

            else:
                print(f"Failed to retrieve details page for {song_info['song']}. Status code: {detail_response.status_code}", '\n', detail_url, '\n', counter)

            # Append the song info to the list
            all_songs.append(song_info)

    with open(f'MusicMetadata_{year}_.csv', mode='w', newline='', encoding='utf-8') as file:
      fieldnames = ['artist', 'song', 'time', 'bpm', 'year', 'genre', 'details', 'album', 'track num']  # Column names for the CSV
      writer = csv.DictWriter(file, fieldnames=fieldnames)

      writer.writeheader()  # Write the header row
      writer.writerows(all_songs)  # Write the song data

    print(f"Data has been written to 'songs_data.csv'.")

# Display the first 10 songs from all years
if all_songs:
    for song in all_songs[:10]:
        print(song)
else:
    print("No songs retrieved.")


Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST', 'TITLE', 'TIME', 'BPM', 'YEAR', 'GENRE', 'DISC-TRACK', 'DETAILS']
Data has been written to 'songs_data.csv'.
Headers found: ['', 'ARTIST